<a href="https://colab.research.google.com/github/deepakri201/prostateDataCuration/blob/main/Undertanding_Patricks_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code to understand Patrick's dataset, and how many T2/DWI/ADC series are in each study. How to choose the best one for AI models for protate cancer segmentation?

# Parameterization

In [1]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "bwh-at-ncigt-data-m-1641244486" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment

In [7]:
import os
import sys

from google.cloud import bigquery
from google.cloud import storage

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Query and plots

In [8]:
client = bigquery.Client(project=project_name)

query = f"""
-- Find the studies with T2, DWI and ADC
-- To do:
  -- Need to start with patients that have both preop and intraop ...
  -- Array_agg instead of string_agg for ease of viewing
  -- Use regexp instead of LIKE operator
  -- Include OHIF links to each study
  -- Refine rules/simplify for DWI and ADC, examine these separately
  -- Add DCE
-- To think about:
  -- How to choose the "best" T2/DWI/ADC sequence out of the list
----------------------------------------------------------------------

-- Get the distinct studyUIDs, and the possible SeriesDescriptions
-- for t2 and ax
WITH select_T2_axial_studies AS(

  SELECT
    DISTINCT(StudyInstanceUID) as StudyInstanceUID,
    STRING_AGG(DISTINCT(SeriesDescription)) as SeriesDescription_T2_AX,
    COUNT(DISTINCT(SeriesDescription)) as num_t2_ax_series
  FROM
    `bwh-at-ncigt-data-m-1641244486.mgb_cre3_irb2020p003809.dicom_metadata_March2023`
  WHERE
    LOWER(SeriesDescription) LIKE "%t2%" AND
    LOWER(SeriesDescription) LIKE "%ax%" AND
    LOWER(SeriesDescription) NOT LIKE "%diff%" AND
    LOWER(SeriesDescription) NOT LIKE "%guidance%" AND
    LOWER(SeriesDescription) NOT LIKE "%guidence%" AND
    LOWER(SeriesDescription) NOT LIKE "%template%"
  GROUP BY
    StudyInstanceUID
  ORDER BY
    num_t2_ax_series DESC

),

-- Select the DWI series that also have T2
-- Make sure "apparent" or "coefficient" are not in the names for the DWI sequences
select_DWI_studies AS(
  SELECT
    DISTINCT(select_T2_axial_studies.StudyInstanceUID),
    select_T2_axial_studies.SeriesDescription_T2_AX,
    select_T2_axial_studies.num_t2_ax_series,
    STRING_AGG(DISTINCT(dicom_metadata.SeriesDescription)) as SeriesDescription_DWI,
    COUNT(DISTINCT(dicom_metadata.SeriesDescription)) as num_dwi_series
  FROM
    select_T2_axial_studies
  JOIN
    `bwh-at-ncigt-data-m-1641244486.mgb_cre3_irb2020p003809.dicom_metadata_March2023` AS dicom_metadata
  ON
    select_T2_axial_studies.StudyInstanceUID = dicom_metadata.StudyInstanceUID
  WHERE
    (LOWER(dicom_metadata.SeriesDescription) LIKE "%dwi%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%apparent%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%adc%" AND
    UPPER(dicom_metadata.SeriesDescription) NOT LIKE "%ADC%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%trace%" AND
    UPPER(dicom_metadata.SeriesDescription) NOT LIKE "%TRACE%")
    OR
    (LOWER(dicom_metadata.SeriesDescription) LIKE "%diffusion%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%apparent%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%adc%" AND
    UPPER(dicom_metadata.SeriesDescription) NOT LIKE "%ADC%" AND
    LOWER(dicom_metadata.SeriesDescription) NOT LIKE "%trace%" AND
    UPPER(dicom_metadata.SeriesDescription) NOT LIKE "%TRACE%")
  GROUP BY
    select_T2_axial_studies.StudyInstanceUID,
    select_T2_axial_studies.SeriesDescription_T2_AX,
    select_T2_axial_studies.num_t2_ax_series
)

-- Select the ADC series that also have T2 and DWI
SELECT
  DISTINCT(select_DWI_studies.StudyInstanceUID),
  select_DWI_studies.SeriesDescription_T2_AX,
  select_DWI_studies.num_t2_ax_series,
  select_DWI_studies.SeriesDescription_DWI,
  select_DWI_studies.num_dwi_series,
  STRING_AGG(DISTINCT(dicom_metadata.SeriesDescription)) as SeriesDescription_ADC,
  COUNT(DISTINCT(dicom_metadata.SeriesDescription)) as num_adc_series,
  ANY_VALUE(CONCAT("https://idc-tester-dk.web.app/projects/bwh-at-ncigt-data-m-1641244486/locations/us-central1/datasets/mgb-cre3-irb2020p003809/dicomStores/dicom-March2023/study/", select_DWI_studies.StudyInstanceUID)) as viewer_url
FROM
  select_DWI_studies
JOIN
  `bwh-at-ncigt-data-m-1641244486.mgb_cre3_irb2020p003809.dicom_metadata_March2023` AS dicom_metadata
ON
  select_DWI_studies.StudyInstanceUID = dicom_metadata.StudyInstanceUID
WHERE
  ((LOWER(dicom_metadata.SeriesDescription) LIKE "%apparent%" AND LOWER(dicom_metadata.SeriesDescription) LIKE "%coefficient%") OR
  LOWER(dicom_metadata.SeriesDescription) LIKE "%adc" OR
  UPPER(dicom_metadata.SeriesDescription) LIKE "%ADC%") # AND
  # num_t2_ax_series > 1
GROUP BY
  select_DWI_studies.StudyInstanceUID,
  select_DWI_studies.SeriesDescription_T2_AX,
  select_DWI_studies.num_t2_ax_series,
  select_DWI_studies.SeriesDescription_DWI,
  select_DWI_studies.num_dwi_series
ORDER BY
  select_DWI_studies.num_t2_ax_series DESC;
  """


job_config = bigquery.QueryJobConfig()
df = client.query(query, job_config=job_config).to_dataframe()

Forbidden: ignored